In [1]:
data_dir = '/home/ubuntu/data/iceberg'

import pandas as pd
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Dropout, Flatten, Input, Conv2D, Cropping2D
from keras.layers import MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation
from keras.layers.merge import Add, Concatenate
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
import cv2
import keras
import os
from keras.callbacks import TensorBoard

from keras import __version__
print(__version__)


Using TensorFlow backend.


2.1.1


In [2]:
np.random.seed(1234)

def get_scaled_imgs(df):
    imgs = []

    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)

        # Rescale
        a = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
        b = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())
        c = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())

#         imgs.append(np.dstack((band_1, band_2, band_3)))
        imgs.append(np.dstack((a, b, c)))

    return np.array(imgs)

def get_more_images(imgs):
    
    more_images = []
    vert_flip_imgs = []
    hori_flip_imgs = []
      
    for i in range(0,imgs.shape[0]):
        a=imgs[i,:,:,0]
        b=imgs[i,:,:,1]
        c=imgs[i,:,:,2]
        
        av=cv2.flip(a,1)
        ah=cv2.flip(a,0)
        bv=cv2.flip(b,1)
        bh=cv2.flip(b,0)
        cv=cv2.flip(c,1)
        ch=cv2.flip(c,0)
        
        vert_flip_imgs.append(np.dstack((av, bv, cv)))
        hori_flip_imgs.append(np.dstack((ah, bh, ch)))
      
    v = np.array(vert_flip_imgs)
    h = np.array(hori_flip_imgs)
       
    more_images = np.concatenate((imgs,v,h))
    
    return more_images

In [3]:
df_train = pd.read_json(os.path.join(data_dir, 'train.json'))
Xtrain = get_scaled_imgs(df_train)
Ytrain = np.array(df_train['is_iceberg'])

df_train.inc_angle = df_train.inc_angle.replace('na',0)
idx_tr = np.where(df_train.inc_angle>0)

Ytrain = Ytrain[idx_tr[0]]
Xtrain = Xtrain[idx_tr[0],...]
Xinc = df_train.inc_angle[idx_tr[0]]

In [6]:
Xtrain[1,:,:,1].max()

0.72976856454753891

In [8]:
# Create a train and validation split, 75% of data used in training
from sklearn.model_selection import train_test_split

X_train, X_valid, X_inc_train, X_inc_valid, Y_train, Y_valid = train_test_split(Xtrain,
                                    Xinc, Ytrain, random_state=666, train_size=0.9, test_size=0.1)

X_train.shape, X_valid.shape, X_angle_train.shape, X_angle_valid.shape, y_train.shape, y_valid.shape

((1323, 75, 75, 3), (148, 75, 75, 3), (1323,), (148,), (1323,), (148,))

In [9]:
X_train = get_more_images(X_train)
X_inc_train = np.concatenate((X_inc_train,X_inc_train,X_inc_train))
Y_train = np.concatenate((Y_train,Y_train,Y_train))

X_train.shape, X_inc_train.shape, Y_train.shape

((3969, 75, 75, 3), (3969,), (3969,))

In [10]:
batch_size = 256
        
tbCallBack = TensorBoard(log_dir='/home/ubuntu/data/tensorboardlogs/', histogram_freq=0, write_graph=True, write_images=True)
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=8, verbose=1, epsilon=1e-4, mode='min')

In [11]:
p = 0.2
input_shape = (75, 75, 3)
num_classes = 2

classifier_input = Input(shape=input_shape)
inc_angle_input = Input(shape=(1,))

# CNN 1
x = Conv2D(64, (3, 3), activation='relu', padding='same')(classifier_input)
x = MaxPooling2D()(x)
x = Dropout(p)(x)

# CNN 2
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)   # REMOVED MAX POOLING FOR VISUALISATION
x = Dropout(p)(x)

# CNN 3
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
x = Dropout(p)(x)

# CNN 3
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
x = Dropout(p)(x)

# x = BatchNormalization(axis=-1)(x)

# CNN 4
x = Conv2D(64,(3,3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
# x = BatchNormalization(axis=-1)(x)
x = Dropout(p)(x)

x = Flatten()(x)

# x = GlobalAveragePooling2D()(x)
m = Concatenate()([inc_angle_input, x])
m = Dense(512, activation='relu')(m)
m = Dense(256, activation='relu')(m)
out = Dense(2, activation='sigmoid')(m)
# out = Activation('softmax')(m)

# optimizer = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
optimizer = Adam(lr=0.0015, decay=0.0)
model = Model(inputs=[classifier_input, inc_angle_input], outputs=out)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [14]:

model.fit([X_train, X_inc_train], to_categorical(Y_train),
                                          batch_size=128, 
                                          epochs=50,
                                          validation_data=([X_valid, X_inc_valid], to_categorical(Y_valid)),
                                          shuffle=True, 
                                          callbacks=[earlyStopping, mcp_save, reduce_lr_loss, tbCallBack])

Train on 3969 samples, validate on 148 samples
Epoch 1/50
3969/3969 [==============================] - 9s 2ms/step - loss: 0.7605 - acc: 0.4961 - val_loss: 0.7908 - val_acc: 0.5135
Epoch 2/50
3969/3969 [==============================] - 6s 2ms/step - loss: 0.7111 - acc: 0.5574 - val_loss: 0.8261 - val_acc: 0.4831
Epoch 3/50
3969/3969 [==============================] - 6s 2ms/step - loss: 0.5430 - acc: 0.7171 - val_loss: 0.5245 - val_acc: 0.7770
Epoch 4/50
3969/3969 [==============================] - 6s 2ms/step - loss: 0.4822 - acc: 0.7754 - val_loss: 0.6037 - val_acc: 0.7635
Epoch 5/50
3969/3969 [==============================] - 6s 2ms/step - loss: 0.4892 - acc: 0.7821 - val_loss: 0.4916 - val_acc: 0.7770
Epoch 6/50
3969/3969 [==============================] - 6s 2ms/step - loss: 0.4403 - acc: 0.8016 - val_loss: 0.4742 - val_acc: 0.7872
Epoch 7/50
3969/3969 [==============================] - 6s 2ms/step - loss: 0.4196 - acc: 0.8185 - val_loss: 0.4496 - val_acc: 0.8007
Epoch 8/50
3969

In [19]:
model.save_weights('/home/ubuntu/data/iceberg/results/weights/norm_image_inc_angle-Copy1_94_acc.h5')

In [66]:
# model.load_weights(filepath = '.mdl_wts.hdf5')
score = model.evaluate([Xtrain,Xinc], to_categorical(Ytrain), verbose=1)
print('Train score:', score[0])
print('Train accuracy:', score[1])

df_test = pd.read_json(os.path.join(data_dir, 'test.json'))
df_test.inc_angle = df_test.inc_angle.replace('na',0)
Xtest = (get_scaled_imgs(df_test))
Xtest_inc = df_test.inc_angle
pred_test = model.predict([Xtest,Xtest_inc])


1471/1471 [==============================] - 1s 638us/step
Train score: 0.126904439938
Train accuracy: 0.956152277362


In [67]:
submission = pd.DataFrame({'id': df_test["id"], 'is_iceberg': pred_test[:,1].reshape((pred_test[:,1].shape[0]))})
print(submission.head(10))

submission.to_csv('/home/ubuntu/data/iceberg/results/final_preds/cnn_model1_aug_' + str(np.around(score[1], decimals=2))  + '.csv', index=False)

         id    is_iceberg
0  5941774d  1.164687e-01
1  4023181e  9.736597e-01
2  b20200e4  5.172412e-05
3  e7f018bb  9.997181e-01
4  4371c8c3  9.265997e-01
5  a8d9b1fd  9.874319e-01
6  29e7727e  2.264617e-01
7  92a51ffb  9.965637e-01
8  c769ac97  1.253932e-05
9  aee0547d  5.985670e-07


In [21]:
# pseudo labelling using the above model

In [38]:
# this is the correct way round with [:,1]
idx_pred_1 = (np.where(pred_test[:,1]>0.95))
idx_pred_0 = (np.where(pred_test[:,1]<0.05))

In [39]:
Xtest_inc.shape, X_inc_train.shape

((8424, 1), (3969, 1))

In [40]:
Xtest_inc = np.expand_dims(Xtest_inc, axis=1)
X_inc_train = np.expand_dims(X_inc_train, axis=1)

In [41]:
Xtrain_pl = np.concatenate((X_train, Xtest[idx_pred_1[0],...], Xtest[idx_pred_0[0],...]))
Ytrain_pl = np.concatenate((Y_train, np.ones(idx_pred_1[0].shape[0]), np.zeros(idx_pred_0[0].shape[0])))
Xinc_pl = np.concatenate((X_inc_train, Xtest_inc[idx_pred_1[0],...], Xtest_inc[idx_pred_0[0],...]))

Xtrain_pl.shape, Ytrain_pl.shape, Xinc_pl.shape

((9763, 75, 75, 3), (9763,), (9763, 1, 1))

In [42]:
Xinc_pl = np.squeeze(Xinc_pl)

In [43]:
p = 0.2
input_shape = (75, 75, 3)
num_classes = 2

classifier_input = Input(shape=input_shape)
inc_angle_input = Input(shape=(1,))

# CNN 1
x = Conv2D(64, (3, 3), activation='relu', padding='same')(classifier_input)
x = MaxPooling2D()(x)
x = Dropout(p)(x)

# CNN 2
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)   # REMOVED MAX POOLING FOR VISUALISATION
x = Dropout(p)(x)

# CNN 3
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
x = Dropout(p)(x)

# CNN 3
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
x = Dropout(p)(x)

# x = BatchNormalization(axis=-1)(x)

# CNN 4
x = Conv2D(64,(3,3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
# x = BatchNormalization(axis=-1)(x)
x = Dropout(p)(x)

x = Flatten()(x)

# x = GlobalAveragePooling2D()(x)
m = Concatenate()([inc_angle_input, x])
m = Dense(512, activation='relu')(m)
m = Dense(256, activation='relu')(m)
out = Dense(2, activation='sigmoid')(m)
# out = Activation('softmax')(m)

# optimizer = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
optimizer = Adam(lr=0.0015, decay=0.0)
model2 = Model(inputs=[classifier_input, inc_angle_input], outputs=out)
model2.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [44]:
model2.fit([Xtrain_pl, Xinc_pl], to_categorical(Ytrain_pl),
                                          batch_size=128, 
                                          epochs=50,
                                          validation_data=([X_valid, X_inc_valid], to_categorical(Y_valid)),
                                          shuffle=True, 
                                          callbacks=[earlyStopping, mcp_save, reduce_lr_loss, tbCallBack])

Train on 9763 samples, validate on 148 samples
Epoch 1/50
9763/9763 [==============================] - 16s 2ms/step - loss: 0.6166 - acc: 0.6713 - val_loss: 0.6229 - val_acc: 0.7128
Epoch 2/50
9763/9763 [==============================] - 15s 2ms/step - loss: 0.3354 - acc: 0.8669 - val_loss: 0.5192 - val_acc: 0.7770
Epoch 3/50
9763/9763 [==============================] - 15s 2ms/step - loss: 0.2501 - acc: 0.9042 - val_loss: 0.4833 - val_acc: 0.7939 loss: 0.2537 - acc
Epoch 4/50
9763/9763 [==============================] - 15s 2ms/step - loss: 0.2342 - acc: 0.9029 - val_loss: 0.2614 - val_acc: 0.8716
Epoch 5/50
9763/9763 [==============================] - 15s 2ms/step - loss: 0.1608 - acc: 0.9301 - val_loss: 0.2916 - val_acc: 0.8649
Epoch 6/50
9763/9763 [==============================] - 15s 2ms/step - loss: 0.1422 - acc: 0.9369 - val_loss: 0.2314 - val_acc: 0.8986
Epoch 7/50
9763/9763 [==============================] - 15s 2ms/step - loss: 0.1399 - acc: 0.9404 - val_loss: 0.2208 - val_a

In [ ]:
# as well as adding the pseudo labelling, augment the pseudo labelled data?

In [45]:
# this way will reaugment the orginal images, perhaps doesn't matter? 

Xtrain_pl = get_more_images(Xtrain_pl)
Xinc_pl = np.concatenate((Xinc_pl,Xinc_pl,Xinc_pl))
Ytrain_pl = np.concatenate((Ytrain_pl,Ytrain_pl,Ytrain_pl))

Xtrain_pl.shape, Ytrain_pl.shape, Xinc_pl.shape

((29289, 75, 75, 3), (29289,), (29289,))

In [46]:
model2.fit([Xtrain_pl, Xinc_pl], to_categorical(Ytrain_pl),
                                          batch_size=128, 
                                          epochs=50,
                                          validation_data=([X_valid, X_inc_valid], to_categorical(Y_valid)),
                                          shuffle=True, 
                                          callbacks=[earlyStopping, mcp_save, reduce_lr_loss, tbCallBack])

Train on 29289 samples, validate on 148 samples
Epoch 1/50
29289/29289 [==============================] - 46s 2ms/step - loss: 0.0408 - acc: 0.9864 - val_loss: 0.1343 - val_acc: 0.9561
Epoch 2/50
29289/29289 [==============================] - 45s 2ms/step - loss: 0.0367 - acc: 0.9876 - val_loss: 0.1561 - val_acc: 0.9459
Epoch 3/50
29289/29289 [==============================] - 45s 2ms/step - loss: 0.0316 - acc: 0.9882 - val_loss: 0.1464 - val_acc: 0.9662
Epoch 4/50
29289/29289 [==============================] - 45s 2ms/step - loss: 0.0343 - acc: 0.9881 - val_loss: 0.1358 - val_acc: 0.9595 acc: 0.98 - ETA: 5s - loss: 0.0344 - acc: 0.988 - ETA: 4s - loss: 0.0344 - acc: 0.987 - ETA: 4s - lo
Epoch 5/50
29289/29289 [==============================] - 46s 2ms/step - loss: 0.0291 - acc: 0.9892 - val_loss: 0.1477 - val_acc: 0.9595
Epoch 6/50
29289/29289 [==============================] - 45s 2ms/step - loss: 0.0299 - acc: 0.9893 - val_loss: 0.1663 - val_acc: 0.9527
Epoch 7/50
29289/29289 [=====

In [65]:
model2.save_weights('/home/ubuntu/data/iceberg/results/weights/norm_image_inc_angle-Copy1_model2_96_acc.h5')

In [68]:
# model.load_weights(filepath = '.mdl_wts.hdf5')
score = model2.evaluate([Xtrain,Xinc], to_categorical(Ytrain), verbose=1)
print('Train score:', score[0])
print('Train accuracy:', score[1])

df_test = pd.read_json(os.path.join(data_dir, 'test.json'))
df_test.inc_angle = df_test.inc_angle.replace('na',0)
Xtest = (get_scaled_imgs(df_test))
Xtest_inc = df_test.inc_angle
pred_test = model.predict([Xtest,Xtest_inc])

1471/1471 [==============================] - 1s 645us/step
Train score: 0.0224028783765
Train accuracy: 0.99388171312


In [69]:
submission = pd.DataFrame({'id': df_test["id"], 'is_iceberg': pred_test[:,1].reshape((pred_test[:,1].shape[0]))})
print(submission.head(10))

submission.to_csv('/home/ubuntu/data/iceberg/results/final_preds/cnn_model2_pseudo_' + str(np.around(score[1], decimals=2))  + '.csv', index=False)

         id    is_iceberg
0  5941774d  1.164687e-01
1  4023181e  9.736597e-01
2  b20200e4  5.172412e-05
3  e7f018bb  9.997181e-01
4  4371c8c3  9.265997e-01
5  a8d9b1fd  9.874319e-01
6  29e7727e  2.264617e-01
7  92a51ffb  9.965637e-01
8  c769ac97  1.253932e-05
9  aee0547d  5.985670e-07


In [50]:
Xtest.shape, Xtest_inc.shape, pred_test.shape

((8424, 75, 75, 3), (8424,), (8424, 2))

In [51]:
Xtest = get_more_images(Xtest)
Xtest_inc = np.concatenate((Xtest_inc,Xtest_inc,Xtest_inc))
pred_test = np.concatenate((pred_test, pred_test, pred_test))

Xtest.shape, Xtest_inc.shape, pred_test.shape

((25272, 75, 75, 3), (25272,), (25272, 2))

In [52]:
# this is the correct way round with [:,1]
idx_pred_1 = (np.where(pred_test[:,1]>0.95))
idx_pred_0 = (np.where(pred_test[:,1]<0.05))

In [57]:
# Xtest_inc = np.expand_dims(Xtest_inc, axis=1)
X_inc_train = np.expand_dims(X_inc_train, axis=1)

In [58]:
# X_inc_train = np.squeeze(X_inc_train)
X_inc_train.shape

(3969, 1)

In [59]:
Xtrain_pl = np.concatenate((X_train, Xtest[idx_pred_1[0],...], Xtest[idx_pred_0[0],...]))
Ytrain_pl = np.concatenate((Y_train, np.ones(idx_pred_1[0].shape[0]), np.zeros(idx_pred_0[0].shape[0])))
Xinc_pl = np.concatenate((X_inc_train, Xtest_inc[idx_pred_1[0],...], Xtest_inc[idx_pred_0[0],...]))

Xtrain_pl.shape, Ytrain_pl.shape, Xinc_pl.shape

((21351, 75, 75, 3), (21351,), (21351, 1))

In [63]:
p = 0.4
input_shape = (75, 75, 3)
num_classes = 2

classifier_input = Input(shape=input_shape)
inc_angle_input = Input(shape=(1,))

# CNN 1
x = Conv2D(64, (3, 3), activation='relu', padding='same')(classifier_input)
x = MaxPooling2D()(x)
x = Dropout(p)(x)

# CNN 2
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)   # REMOVED MAX POOLING FOR VISUALISATION
x = Dropout(p)(x)

# CNN 3
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
x = Dropout(p)(x)

# CNN 3
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
x = Dropout(p)(x)

# x = BatchNormalization(axis=-1)(x)

# CNN 4
x = Conv2D(64,(3,3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
# x = BatchNormalization(axis=-1)(x)
x = Dropout(p/2)(x)

x = Flatten()(x)

# x = GlobalAveragePooling2D()(x)
m = Concatenate()([inc_angle_input, x])
m = Dense(512, activation='relu')(m)
m = Dense(256, activation='relu')(m)
out = Dense(2, activation='sigmoid')(m)
# out = Activation('softmax')(m)

# optimizer = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
optimizer = Adam(lr=0.0015, decay=0.0)
model3 = Model(inputs=[classifier_input, inc_angle_input], outputs=out)
model3.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [70]:
model3.fit([Xtrain_pl, Xinc_pl], to_categorical(Ytrain_pl),
                                          batch_size=256, 
                                          epochs=50,
                                          validation_data=([X_valid, X_inc_valid], to_categorical(Y_valid)),
                                          shuffle=True, 
                                          callbacks=[earlyStopping, mcp_save, reduce_lr_loss, tbCallBack])

Train on 21351 samples, validate on 148 samples
Epoch 1/50
21351/21351 [==============================] - 32s 2ms/step - loss: 0.5016 - acc: 0.7432 - val_loss: 0.6544 - val_acc: 0.7973
Epoch 2/50
21351/21351 [==============================] - 32s 1ms/step - loss: 0.1927 - acc: 0.9379 - val_loss: 0.4042 - val_acc: 0.8716
Epoch 3/50
21351/21351 [==============================] - 32s 1ms/step - loss: 0.1396 - acc: 0.9557 - val_loss: 0.3343 - val_acc: 0.8784
Epoch 4/50
21351/21351 [==============================] - 32s 1ms/step - loss: 0.0918 - acc: 0.9642 - val_loss: 0.1993 - val_acc: 0.9189
Epoch 5/50
21351/21351 [==============================] - 32s 1ms/step - loss: 0.0761 - acc: 0.9714 - val_loss: 0.2107 - val_acc: 0.9122
Epoch 6/50
21351/21351 [==============================] - 32s 1ms/step - loss: 0.0686 - acc: 0.9722 - val_loss: 0.1700 - val_acc: 0.9122
Epoch 7/50
21351/21351 [==============================] - 32s 1ms/step - loss: 0.0643 - acc: 0.9747 - val_loss: 0.2037 - val_acc: 

In [71]:
model3.save_weights('/home/ubuntu/data/iceberg/results/weights/norm_image_inc_angle-Copy1_model3_95_acc.h5')

In [72]:
# model.load_weights(filepath = '.mdl_wts.hdf5')
score = model3.evaluate([Xtrain,Xinc], to_categorical(Ytrain), verbose=1)
print('Train score:', score[0])
print('Train accuracy:', score[1])

df_test = pd.read_json(os.path.join(data_dir, 'test.json'))
df_test.inc_angle = df_test.inc_angle.replace('na',0)
Xtest = (get_scaled_imgs(df_test))
Xtest_inc = df_test.inc_angle
pred_test = model.predict([Xtest,Xtest_inc])

1471/1471 [==============================] - 1s 646us/step
Train score: 0.110659150335
Train accuracy: 0.956492182189


In [106]:
# submit_nums = np.clip(pred_test[:,1], 0.1, 0.9)
submit_nums[1]

0.89999998

In [73]:
submission = pd.DataFrame({'id': df_test["id"], 'is_iceberg': pred_test[:,1].reshape((pred_test[:,1].shape[0]))})
print(submission.head(10))

submission.to_csv('/home/ubuntu/data/iceberg/results/final_preds/cnn_model3_pseudo_aug_' + str(np.around(score[1], decimals=2))  + '.csv', index=False)

         id    is_iceberg
0  5941774d  1.164687e-01
1  4023181e  9.736597e-01
2  b20200e4  5.172412e-05
3  e7f018bb  9.997181e-01
4  4371c8c3  9.265997e-01
5  a8d9b1fd  9.874319e-01
6  29e7727e  2.264617e-01
7  92a51ffb  9.965637e-01
8  c769ac97  1.253932e-05
9  aee0547d  5.985670e-07


In [92]:
df_test.shape

(8424, 4)

In [114]:
model.evaluate([Xtrain,Xinc], to_categorical(Ytrain), verbose=1)

4413/4413 [==============================] - 3s 600us/step


[0.13649047835023664, 0.95014729196998826]